### modeling

In [ ]:
# import statements
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
import sklearn.metrics as metrics

In [ ]:
X_test_30_transform = pd.read_pickle("data/X_test_30_transform.pkl")

In [ ]:
X_train_30_transform = pd.read_pickle("data/X_train_30_transform.pkl")

In [ ]:
y_train_30 = pd.read_pickle("data/y_train_30.pkl")

In [ ]:
y_test_30 = pd.read_pickle("data/y_test_30.pkl")

### resampling

In [ ]:
rus = RandomUnderSampler(sampling_strategy = 0.01, random_state=42)


In [ ]:
X_train_transform_resampled, y_train_resampled = rus.fit_resample(X_train_30_transform, y_train_30)

### classifying

#### stochastic gradient descent

In [ ]:
SGDC = SGDClassifier(loss= "log_loss", warm_start=True, alpha = 0.0001, penalty= 'l2')

In [ ]:
list_df = [X_train_30_transform[i:i+50000] for i in range(0,len(X_train_30_transform),50000)]


In [ ]:
list_y = [y_train_30[i:i+50000] for i in range(0,len(y_train_30),50000)]

In [ ]:
counter = 1
for df, y in zip(list_df ,list_y):
    SGDC.partial_fit(df, y, classes=np.unique(y_train_30))
    print(f"{counter}. fit done!")
    counter += 1

In [ ]:
y_pred = SGDC.predict(X_test_30_transform)

#### random forest

In [ ]:
rfc = RandomForestClassifier(n_estimators=250, max_depth = 40, n_jobs = -1, random_state = 42)

In [ ]:
rfc.fit(X_train_30_transform, y_train_30, )

In [ ]:
rfc_pred = rfc.predict(X_test_30_transform)

#### random forest undersampled

In [ ]:
rfc_sampled = RandomForestClassifier(n_estimators=250, max_depth = 40, n_jobs = -1, random_state = 42, )

In [ ]:
rfc_sampled.fit(X_train_transform_resampled, y_train_resampled)

In [ ]:
rfc_pred_sampled_pred = rfc_sampled.predict(X_test_30_transform)

#### results

In [ ]:
print("stochastic gradient descent")
print("accuracy: ",metrics.accuracy_score(y_test_30, y_pred))
print("recall: ",metrics.recall_score(y_test_30, y_pred))
print("F1: ",metrics.f1_score(y_test_30, y_pred))
metrics.confusion_matrix(y_test_30, y_pred)

In [ ]:
print("random forest")
print("accuracy: ",metrics.accuracy_score(y_test_30, rfc_pred))
print("recall: ",metrics.recall_score(y_test_30, rfc_pred))
print("F1: ",metrics.f1_score(y_test_30, rfc_pred))
metrics.confusion_matrix(y_test_30, rfc_pred)

In [ ]:
print("random forest undersampled")
print("accuracy: ",metrics.accuracy_score(y_test_30, rfc_pred_sampled_pred))
print("recall: ",metrics.recall_score(y_test_30, rfc_pred_sampled_pred))
print("F1: ",metrics.f1_score(y_test_30, rfc_pred_sampled_pred))
metrics.confusion_matrix(y_test_30, rfc_pred_sampled_pred)